In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import RMSprop
from keras.datasets import mnist
from keras.utils import np_utils
from keras import initializers
from keras import backend as K
from keras.layers import ELU


In [2]:
batch_size = 128
nb_classes = 10
epochs = 3

img_rows, img_cols = 28, 28         # input image dimensions
pool_size = (2, 2)                  # size of pooling area for max pooling
prob_drop_conv = 0.2                # drop probability for dropout @ conv layer
prob_drop_hidden = 0.5              # drop probability for dropout @ fc layer


In [3]:
def init_weights(shape, name=None):
    return initializations.normal(shape, scale=0.01, name=name)

# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print('X_train original shape:', X_train.shape)

#if K.backend.image_dim_ordering() == 'th':
#    # For Theano backend
#    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
#    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
#    input_shape = (1, img_rows, img_cols)
#else:
# For TensorFlow backend
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)
    
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')


X_train original shape: (60000, 28, 28)
X_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [6]:
# Convolutional model
model = Sequential()

# conv1 layer
model.add(Conv2D(32, (3,3), padding='same', activation='relu')) # input_shape=input_shape, init=init_weights))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), padding='same'))
model.add(Dropout(prob_drop_conv))

# conv2 layer
model.add(Conv2D(64, (3,3), padding='same', activation='relu')) # , init=init_weights
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), padding='same'))
model.add(Dropout(prob_drop_conv))

# conv3 layer
model.add(Conv2D(128, (3,3), padding='same', activation='relu')) # , init=init_weights
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), padding='same'))
model.add(Flatten())
model.add(Dropout(prob_drop_conv))

# fc1 layer
#model.add(Dense(625, activation='relu')) # , init=init_weights
model.add(Dense(625, activation='linear')) # , init=init_weights
model.add(ELU())
model.add(Dropout(prob_drop_hidden))

# fc2 layer
model.add(Dense(10, activation='softmax')) # , init=init_weights

opt = RMSprop(learning_rate=0.001, rho=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
#model.summary()


In [7]:
# Train
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, shuffle=True, verbose=1)


Epoch 1/3
469/469 [==============================] - 12s 23ms/step - loss: 0.2262 - accuracy: 0.9271
Epoch 2/3
469/469 [==============================] - 11s 23ms/step - loss: 0.0650 - accuracy: 0.9798
Epoch 3/3
469/469 [==============================] - 11s 23ms/step - loss: 0.0486 - accuracy: 0.9854


In [8]:
# Evaluate
evaluation = model.evaluate(X_test, Y_test, batch_size=256, verbose=1)
print('Summary: Loss over the test dataset: %.4f, Accuracy: %.4f' % (evaluation[0], evaluation[1]))


40/40 [==============================] - 1s 15ms/step - loss: 0.0251 - accuracy: 0.9914
Summary: Loss over the test dataset: 0.0251, Accuracy: 0.9914
